In [4]:
import numpy as np
import pandas as pd
import math
from collections import Counter

# Build Tree
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    # By default, we will set the split_criteria to "Information Gain" and the max_depth to be 100
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None, split_criteria="Information Gain"):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.split_criteria=split_criteria
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    # Building subtrees (branches)
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)

        return Node(best_feature, best_thresh, left, right)

    # Determine best split bbased on split criteria
    def _best_split(self, X, y, feat_idxs):
      best_gain = -1
      split_idx, split_threshold = None, None

      if self.split_criteria == "Information Gain":
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            unique_columns = np.unique(X_column)

            for column in unique_columns:
                # calculate the information gain
                gain = self._information_gain(y, X_column, column)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = column

        return split_idx, split_threshold
      if self.split_criteria == "Gain Ratio":
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            unique_columns = np.unique(X_column)
            
            for column in unique_columns:
                split_info = 0

                # calculate split info
                count = Counter(X_column)
                prob = count[column] / float(len(X_column))
                split_info -= prob * math.log2(prob)

                # calculate the information gain
                gain = self._information_gain(y, X_column, column)

                # calculate gain ratio
                if split_info != 0:
                    gain_ratio = gain / split_info

                    if gain_ratio > best_gain:
                        best_gain = gain_ratio
                        split_idx = feat_idx
                        split_threshold = column

        return split_idx, split_threshold
      if self.split_criteria == "Gini Index":
        best_gain = 1
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            unique_columns = np.unique(X_column)

            for column in unique_columns:
                # calculate the gini index
                gini = self._gini_index(y, X_column, column)
                if gini < best_gain:
                    best_gain = gini
                    split_idx = feat_idx
                    split_threshold = column
    
        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        # parent entropy = base entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)

        left_indexes_length, right_indexes_length = len(left_idxs), len(right_idxs)
        left_indexes_entropy, right_indexes_entropy = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (left_indexes_length/n) * left_indexes_entropy + (right_indexes_length/n) * right_indexes_entropy

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _gini_index(self, y, X_column, threshold):
        value_counts = Counter(X_column)
        totalLength = len(X_column)
        p_sum = 0

        for key in value_counts.keys():
            p_sum = p_sum  +  (value_counts[key] / totalLength ) * (value_counts[key] / totalLength ) 
        gini = 1 - p_sum
        return gini

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    # Calculate Entropy
    def _entropy(self, data):
        # The entropy (uncertainty) of getting a value randomly from the dataset will be higher if the dataset is impure, 
        # i.e. uneven number of males/females in a dataset

        # The higher the impurity in a dataset, the higher the entropy, vice versa.
        total = len(data)
        count = Counter(data)
        ent = 0

        for label in count:
            prob = count[label] / float(total)
            ent -= prob * math.log2(prob)

        return ent

    def _most_common_label(self, y):
        if y.size == 0:
            return 0

        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        

## Now we will add a method called createDataSet which will be used to generate our training and testing data sets.

In [5]:
# Load and Process Data
def createDataSet(target=""):
    # Import base data in from adult.data
    base_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income (class)']
    data = pd.read_csv('/adult.data', names=base_labels)
    
    # Rearrange the columns to have age at the end for classification
    labels = base_labels

    # Allow the ability to change the target column
    if target != "":
      labels.append(labels.pop(labels.index(target)))
      data = data[labels]

    # Handle missing values
    data = data.fillna(0)  # Fill missing values with 0

    # Handle invalid data
    data = data.replace(-1, 0)  # Replace invalid values (-1) with 0
    
    # Randomly shuffle the data
    data = data.sample(frac=1)

    # Reset the index
    data = data.reset_index(drop=True)

    # Calculate the index to split the data
    split_index = int(len(data) * 0.6)

    # Split the data into training and test sets
    train_data = data.iloc[:split_index, :]
    train_data_target = train_data[train_data.columns[-1]]
    train_data = train_data.iloc[: , :-1]
    
    test_data = data.iloc[:split_index, :]
    test_data_target = test_data[test_data.columns[-1]]
    test_data = test_data.iloc[: , :-1]

    # Remove last column from labels (for classification)
    return train_data.to_numpy(), train_data_target.to_numpy(), test_data.to_numpy(), test_data_target.to_numpy()

## We will now start to build and train the decision trees based on the 3 different split criterias namely, Information Gain (dt1), Gain Ratio (dt2) and Gini Index (dt3).

## To further improve the training speed while maintaing the model's accuracy, I have added a max_depth feature which is set to a default value of 100, as well as a min_samples_split feature which is set to a default value of 2

In [6]:
import copy

train_data, train_data_target, test_data, test_data_target = createDataSet()

dt1 = DecisionTree(max_depth=10)
dt2 = DecisionTree(max_depth=10, split_criteria="Gain Ratio")
dt3 = DecisionTree(max_depth=10, split_criteria="Gini Index")

dt1.fit(copy.deepcopy(train_data), copy.deepcopy(train_data_target))
dt2.fit(copy.deepcopy(train_data), copy.deepcopy(train_data_target))
dt3.fit(copy.deepcopy(train_data), copy.deepcopy(train_data_target))

## Now we will test the decision trees and store their predictions for analysis. We will use the accuracy method, to determine the accuracy of each decision tree's prediction

In [9]:
predictions1 = dt1.predict(copy.deepcopy(test_data))
predictions2 = dt2.predict(copy.deepcopy(test_data))
predictions3 = dt3.predict(copy.deepcopy(test_data))

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

info_gain_acc = accuracy(test_data_target, predictions1)
print(info_gain_acc)

gain_ratio_acc = accuracy(test_data_target, predictions2)
print(gain_ratio_acc)

gini_index_acc = accuracy(test_data_target, predictions3)
print(gini_index_acc)

0.8667076167076168
0.8263206388206388
0.7556818181818182


## Comparing the 3 different split criterias, we can see that **Information Gain has the highest accuracy at 86.67%**, with Gain Ratio at second with an accuracy of 82.63% while Gini Index trails last at 75.57%.

In [8]:
# Define a function to return the majority vote
def ensemble_predict(dt1, dt2, dt3, x):
    predictions = [predictions1, predictions2, predictions3]
    return [Counter(x).most_common(1)[0][0] for x in zip(*predictions)]

# Use the ensemble model to predict on new data
ensemble_prediction = ensemble_predict(dt1, dt2, dt3, test_data_target)

ensemble_acc = accuracy(test_data_target, ensemble_prediction)
print(ensemble_acc)

0.8263206388206388


## As we can see, the ensemble method achieved an accuracy of 82.63%. In comparison with the highest accuracy achieved without the ensemble method, we can see that using Information Gain alone has resulted in a higher accuracy.